<a href="https://colab.research.google.com/github/magnujo/ModuleViewGenerator/blob/main/Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
!{sys.executable} -m pip install gitpython
!{sys.executable} -m pip install pyvis

In [ ]:
# all the needed imports
import os
import sys
import re

import pathlib
from pathlib import Path
import pprint

import networkx as nx
import matplotlib.pyplot as plt

from git import Repo



In [ ]:
cwd = os.getcwd()
print(cwd)


/content


In [ ]:
# NOTE: this must end in /
CODE_ROOT_FOLDER="/content/Zeeguu-API/"


In [ ]:
if not os.path.exists(CODE_ROOT_FOLDER):
  Repo.clone_from("https://github.com/zeeguu-ecosystem/Zeeguu-API", CODE_ROOT_FOLDER)


In [ ]:
repo = Repo(CODE_ROOT_FOLDER)

In [ ]:
#Function by Mircea Lungu
def file_path(file_name):
    return CODE_ROOT_FOLDER+file_name

In [ ]:
#Function by Mircea Lungu
def imports(file):

    def extract_import_from_line(line):
      x = re.search("^import (\S+)", line) 
      x = re.search("^from (\S+)", line) 
      return x.group(1)

    lines = [line for line in open(file)]
    
    all_imports = []
    for line in lines:
        try:
            all_imports.append(extract_import_from_line(line))
        except:
            continue

    return all_imports

assert "sqlalchemy" in imports(file_path('zeeguu_core/model/user.py'))

In [ ]:
#Function by Mircea Lungu
def module_from_file_path(folder_prefix, full_path):
    # extracting a module from a file name
    # e.g. ../zeeguu_core/model/user.py -> zeeguu_core.model.user
    
    file_name = full_path[len(folder_prefix):]
    file_name = file_name.replace("/",".")
    file_name = file_name.replace(".py","")
    return file_name

#Function by Mircea Lungu
def module(full_path):
    return module_from_file_path(CODE_ROOT_FOLDER, full_path)

assert 'zeeguu_core.model.user' == module(file_path('zeeguu_core/model/user.py'))

In [ ]:
#Function by Mircea Lungu
def top_level_module(module_name, depth=1):
    components = module_name.split(".")
    return ".".join(components[:depth])

assert (top_level_module("zeeguu_core.model.util") == "zeeguu_core")
assert (top_level_module("zeeguu_core.model.util", 2) == "zeeguu_core.model")

In [ ]:
#Function by Magnus Johannsen
def draw_graph_with_labels(G, node_size=500, layout="none", figsize=(15,15)):
    plt.figure(figsize=figsize)
    layouts = {
        "circular": nx.draw_circular, 
        "none": nx.draw,
        "spring": nx.draw_spring, 
        "random": nx.draw_random,
        "kamada_kawai": nx.draw_kamada_kawai,
        "planar": nx.draw_planar,
        "shell": nx.draw_shell,
        "spectral": nx.draw_spectral         
        }
    layouts[layout](G, with_labels=True, node_size=node_size)
     
    plt.show()

In [ ]:
#Function by Mircea Lungu
def is_system_module(m):
    return m.startswith('zeeguu') or m=='tools' or m=="tests" 

In [ ]:
#Function by Mircea Lungu
def dependencies_graph(sourcepath):
    files = Path(sourcepath).rglob("*.py")
    
    G = nx.DiGraph()

    for file in files:
        m = module(str(file))
      
        if m not in G.nodes:
            G.add_node(m)

        for each in imports(str(file)):
            G.add_edge(m, each)

    return G

In [ ]:
#Function by Mircea Lungu
def abstracted_to_level(G, level):
    aG = nx.DiGraph()
    for each in G.edges():
        source = top_level_module(each[0], depth=level)
        destination = top_level_module(each[1], depth=level)
        if is_system_module(source) and is_system_module(destination):
            aG.add_edge(source, destination)
    return aG   

In [ ]:
#Function by Magnus Johannsen
def modules_to_numbers(G):
  mods_to_nums = {}
  nums_to_mods = {}
  for count, node in enumerate(G.nodes()):
    mods_to_nums[node] = count
    nums_to_mods[count] = node
  
  return mods_to_nums, nums_to_mods


In [ ]:
#Function by Mircea Lungu
def LOC(file):
    return sum([1 for line in open(file)])

#Function by Mircea Lungu
def module_size(m):
    size = 0
    files = Path(CODE_ROOT_FOLDER).rglob("*.py")
    for file in files:
        module_name = module(str(file))
        if module_name.startswith(m + '.'):
            size += LOC(str(file))
            
    return size

10478

In [ ]:
#Function by Magnus Johannsen
def draw_graph_with_numbers(G, converter, layout="none", figsize=(15,15), weight_function=None):

    plt.figure(figsize=figsize)
    layouts = {
        "circular": nx.draw_circular, 
        "none": nx.draw,
        "spring": nx.draw_spring, 
        "random": nx.draw_random,
        "kamada_kawai": nx.draw_kamada_kawai,
        "planar": nx.draw_planar,
        "shell": nx.draw_shell,
        "spectral": nx.draw_spectral         
        }
    #nx.draw(G, pos, with_labels=True)
    if weight_function is not None:
      node_weights = [weight_function(converter[each]) for each in G.nodes()]   
      layouts[layout](G, with_labels=True, node_size=node_weights)
    else:
      layouts[layout](G, with_labels=True)

    plt.show()

In [ ]:
#Function by Magnus Johannsen
def draw_graph_with_modules(G, layout="none", figsize=(15,15), weight_function=None):

    plt.figure(figsize=figsize)
    layouts = {
        "circular": nx.draw_circular, 
        "none": nx.draw,
        "spring": nx.draw_spring, 
        "random": nx.draw_random,
        "kamada_kawai": nx.draw_kamada_kawai,
        "planar": nx.draw_planar,
        "shell": nx.draw_shell,
        "spectral": nx.draw_spectral         
        }
    if weight_function is not None:
      node_weights = [weight_function(each) for each in G.nodes()]   
      layouts[layout](G, with_labels=True, node_size=node_weights)
    else:
      layouts[layout](G, with_labels=True)

    plt.show()

In [ ]:
#Function by Magnus Johannsen
def get_module_view(sourcepath, graph_layout, graph_size, depth, labeltype="numbers", weight_function=None):
  directed = dependencies_graph(sourcepath)
  directedAbstracted = abstracted_to_level(directed,depth)
  if labeltype == "numbers":
    m_to_n, n_to_m = modules_to_numbers(directedAbstracted)
    pp = pprint.PrettyPrinter(indent=4)
    pp.pprint(n_to_m)
    relabelGraph = nx.relabel.relabel_nodes(directedAbstracted, m_to_n)
    draw_graph_with_numbers(relabelGraph, n_to_m, layout=graph_layout, figsize=graph_size, weight_function=weight_function)
  else:
    draw_graph_with_modules(directedAbstracted, layout=graph_layout, figsize=graph_size, weight_function=weight_function)
